# Cleaning the Groundtruth File
0. Drop all columns where 'num_patches' == NaN
1. Only keep the columns 'PatientID', 'Histologischer Subtyp', 'WSI1', 'WSI2', 'WSI3'
2. All columns with 'Histologischer Subtyp' == 'nicht typisierbar', 'Anaplastisch', 'Desmoplastisch*', 'Desmoplastisch/Anaplastisch', 'Medullomyoblastom', 'Großzellig/anapl.' are thrown out
3. All 'klassisch' and 'Klassisch' values are renamed to CMB
4. All "Extensiv nodulär" and "Desmoplastisch" values are renamed to "DN"
5. For now, drop the columns 'WSI2', 'WSI3'
6. In column 'WSI1', there are sometimes multiple files in one column (they are split by '\n' or ' ), only utilize the first file of them

In [79]:
import pandas as pd
import os

# open xlsx as dataframe and count the all rows
df = pd.read_excel('/home/Maack/Medulloblastoma/mb_anno_070721.xlsx')

# 0.
df_cleaned = df.dropna(subset=['num_patches'])

# 1.
df_cleaned = df_cleaned[['PatientID', 'Histologischer Subtyp', 'WSI1', 'WSI2', 'WSI3']]


# 2.
df_cleaned = df_cleaned[df_cleaned['Histologischer Subtyp'] != 'nicht typisierbar']
df_cleaned = df_cleaned[df_cleaned['Histologischer Subtyp'] != 'Anaplastisch']
df_cleaned = df_cleaned[df_cleaned['Histologischer Subtyp'] != 'Desmoplastisch*']
df_cleaned = df_cleaned[df_cleaned['Histologischer Subtyp'] != 'Desmoplastisch/Anaplastisch']
df_cleaned = df_cleaned[df_cleaned['Histologischer Subtyp'] != 'Medullomyoblastom']
df_cleaned = df_cleaned[df_cleaned['Histologischer Subtyp'] != 'Großzellig/anapl.']

# 3.
df_cleaned['Histologischer Subtyp'] = df_cleaned['Histologischer Subtyp'].replace('klassisch', 'CMB')
df_cleaned['Histologischer Subtyp'] = df_cleaned['Histologischer Subtyp'].replace('Klassisch', 'CMB')

# 4.
df_cleaned['Histologischer Subtyp'] = df_cleaned['Histologischer Subtyp'].replace('Desmoplastisch', 'DN')
df_cleaned = df_cleaned.reset_index(drop=True)

# 5.
df_cleaned = df_cleaned.drop(columns=['WSI2', 'WSI3'])
files = os.listdir("/data/Maack/PANT/ndpi/NDPI")

# 6.
for index, row in df_cleaned.iterrows():
    if row['WSI1'] not in files:
        new_row = row['WSI1'].split('.ndpi')
        # create new df_cleaned with new_row
        df_cleaned.loc[index, 'WSI1'] = f"{new_row[0]}.ndpi"
        
#export df_cleaned as csv
df_cleaned.to_csv('/home/Maack/Medulloblastoma/mb_anno_070721_cleaned.csv', index=False)

## Check if all files in mb_anno_070721_cleaned.csv occur in the NDPI folder 

In [81]:
import pandas as pd
import os

# open xlsx as dataframe and count the all rows
df = pd.read_csv('/home/Maack/Medulloblastoma/mb_anno_070721_cleaned.csv')

files = os.listdir("/data/Maack/PANT/ndpi/NDPI")


counter = 0
# check if all df[WSI1] are in /data/Maack/PANT/ndpi/NDPI
for index, row in df.iterrows():
    if row['WSI1'] not in files:
        counter += 1
        print(row['WSI1'])

print(counter)


0


## Delete all files that are not to be found in the ground truth.

In [84]:
# df = pd.read_csv('/home/Maack/Medulloblastoma/mb_anno_070721_cleaned.csv')

files = os.listdir("/data/Maack/PANT/ndpi/NDPI")

# # Delete all files in files that are not in df[WSI1]
# counter = 0
# for file in files:
#     if file not in df['WSI1'].values:
#         os.remove(f"/data/Maack/PANT/ndpi/NDPI/{file}")
#         counter += 1

# print(counter)

print(len(files))

161
